<a href="https://colab.research.google.com/github/ermilovim/ml_kaggle/blob/main/house_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обучение



In [115]:
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, scale
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor,RandomForestRegressor,AdaBoostRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import xgboost
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [152]:
train = pd.read_csv('/content/drive/MyDrive/Воробьевы горы/Нейронные сети/house prices/train.csv')
train['From'] = 1
test = pd.read_csv('/content/drive/MyDrive/Воробьевы горы/Нейронные сети/house prices/test.csv')
test['From'] = 0
all = pd.concat([train, test])

In [ ]:
all.head()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

## Работа с датасетом

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(x = df['YearBuilt'], y = df['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('OverallQual', fontsize=13)
plt.xticks(rotation=90)
plt.show()

In [149]:
categorical = []
for label in all:
  if all[label].dtype == 'object':
    categorical.append(label)
print(categorical)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [153]:
all = pd.get_dummies(all, columns=categorical)
# сделали дамми-кодирование, теперь разделяем обратно
df = all[all.From == 1]
df = df.drop('From', axis=1)
test_df = all[all.From == 0]
test_df = test_df.drop(['SalePrice', 'From'], axis=1)

In [33]:
le = LabelEncoder()
ohe = OneHotEncoder(sparse=False)

def do_ohe(df, label, le=le, ohe=ohe):
  df[label] = le.fit_transform(df[label])
  encoded = ohe.fit_transform(df[label].values.reshape(-1, 1))
  tmp = pd.DataFrame(encoded, columns=[label + str(i) for i in range(encoded.shape[1])])
  return tmp


In [61]:
streets = {'Pave': 0, 'Grvl': 1}
df['Street'] = df['Street'].map(streets)

mszoning_tmp = do_ohe(df, 'MSZoning')
df = pd.concat([df, mszoning_tmp], axis=1)
lotshape_tmp = do_ohe(df, 'LotShape')
df = pd.concat([df, lotshape_tmp], axis=1)
utilities_tmp = do_ohe(df, 'Utilities')
df = pd.concat([df, utilities_tmp], axis=1)

In [ ]:
corr_matrix = df.corr()
plt.subplots(figsize=(12, 12))
sns.heatmap(corr_matrix, square=True)


In [ ]:
labels = corr_matrix[corr_matrix['SalePrice']>0.05].index
print(labels)

In [63]:
df = df[corr_matrix[corr_matrix['SalePrice']>0.05].index]

In [154]:
for label in df:
  df[label] = df[label].fillna(df[label].mean())

In [ ]:
df.head()

In [ ]:
df.info()

In [155]:
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

## Разбиваем на train-test и обучаем модели

In [158]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [157]:
X_train = scale(X_train)

In [159]:
log_reg = LogisticRegression(random_state=42)
log_reg.fit(scale(X_train), y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=42)

In [160]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [137]:
xgb = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)

xgb.fit(X_train, y_train)

[13:18:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(colsample_bytree=0.4, learning_rate=0.07, min_child_weight=1.5,
             n_estimators=10000, reg_alpha=0.75, reg_lambda=0.45, seed=42,
             subsample=0.6)

In [138]:
lasso = Lasso(alpha=100, max_iter=10000, random_state=13).fit(X_train,y_train)

In [139]:
ada = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=15),
                  learning_rate=1, n_estimators=100,
                  random_state=13).fit(X_train,y_train)

In [161]:
log_reg_test_preds = log_reg.predict(scale(X_test))
lin_reg_test_preds = lin_reg.predict(X_test)
xgb_test_preds = xgb.predict(X_test)
lasso_test_preds = lasso.predict(X_test)
ada_test_preds = ada.predict(X_test)

## Оценка логистической регрессии

In [141]:
print('MSE on LogReg: {}'.format(
    np.sqrt(mean_squared_error(np.log(y_test),np.log(log_reg_test_preds)))
))
print('R^2 on LogReg: {}'.format(r2_score(y_test, log_reg_test_preds)))

MSE on LogReg: 0.2863819723984677
R^2 on LogReg: 0.5208237432714502


## Оценка линейной регрессии

In [ ]:
print('MSE on LinReg: {}'.format(
    np.sqrt(mean_squared_error(np.log(y_test),np.log(lin_reg_test_preds)))
))
print('R^2 on LinReg: {}'.format(r2_score(y_test, lin_reg_test_preds)))

## Оценка XGBoost

In [172]:
print('MSE on XGBoost: {}'.format(
    np.sqrt(mean_squared_error(np.log(y_test),np.log(xgb_test_preds)))
))
print('R^2 on XGBoost: {}'.format(r2_score(y_test, xgb_test_preds)))

MSE on XGBoost: 0.07377014059495465
R^2 on XGBoost: 0.965354725591028


## Оценка Lasso

In [171]:
print('MSE on LinReg: {}'.format(
    np.sqrt(mean_squared_error(np.log(y_test),np.log(lasso_test_preds)))
))
print('R^2 on LinReg: {}'.format(r2_score(y_test, lasso_test_preds)))

MSE on LinReg: 0.13182438422396414
R^2 on LinReg: 0.9169642514490493


## Оценка AdaBoost

In [145]:
print('MSE on AdaBoost: {}'.format(
    np.sqrt(mean_squared_error(np.log(y_test),np.log(ada_test_preds)))
))
print('R^2 on AdaBoost: {}'.format(r2_score(y_test, ada_test_preds)))

MSE on AdaBoost: 0.1627104336909585
R^2 on AdaBoost: 0.8276072512163134


## Выбор итоговой модели

In [102]:
xgb = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)

xgb.fit(X, y)

[11:54:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(colsample_bytree=0.4, learning_rate=0.07, min_child_weight=1.5,
             n_estimators=10000, reg_alpha=0.75, reg_lambda=0.45, seed=42,
             subsample=0.6)

# Работа с тестовыми данными

In [103]:
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [104]:
ids = test_df['Id']

In [105]:
test_df = pd.get_dummies(test_df, columns=categorical)

In [ ]:
test_df['Street'] = test_df['Street'].map(streets)

mszoning_tmp = do_ohe(test_df, 'MSZoning')
test_df = pd.concat([test_df, mszoning_tmp], axis=1)
lotshape_tmp = do_ohe(test_df, 'LotShape')
test_df = pd.concat([test_df, lotshape_tmp], axis=1)
utilities_tmp = do_ohe(test_df, 'Utilities')
test_df = pd.concat([test_df, utilities_tmp], axis=1)

In [ ]:
test_df = test_df[['LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'ScreenPorch',
       'PoolArea']]

In [106]:
for label in test_df:
  test_df[label] = test_df[label].fillna(df[label].mean())

In [114]:
print(test_df.shape)
print(X.shape)

(1459, 271)
(1460, 289)


In [ ]:
X.head()

In [108]:
test_df.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_RH,...,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Gd,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_Shed,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,882.0,896,0,0,896,0.0,0.0,1,0,2,1,5,0,1961.0,1.0,730.0,140,0,0,0,120,0,0,6,2010,0,0,1,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,1329.0,1329,0,0,1329,0.0,0.0,1,1,3,1,6,0,1958.0,1.0,312.0,393,36,0,0,0,0,12500,6,2010,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,928.0,928,701,0,1629,0.0,0.0,2,1,3,1,6,1,1997.0,2.0,482.0,212,34,0,0,0,0,0,3,2010,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,324.0,926.0,926,678,0,1604,0.0,0.0,2,1,3,1,7,1,1998.0,2.0,470.0,360,36,0,0,0,0,0,6,2010,0,0,0,...,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,1017.0,1280.0,1280,0,0,1280,0.0,0.0,2,0,2,1,5,0,1992.0,2.0,506.0,0,82,0,0,144,0,0,1,2010,0,0,0,...,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [109]:
test_df['Id']

0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64

In [110]:
predictions = xgb.predict(test_df)

ValueError: ignored

In [ ]:
with open('/content/drive/MyDrive/Воробьевы горы/Нейронные сети/house prices/house_submission_5.csv', 'w') as file:
  file.write("Id,SalePrice\n")
  for i in range(len(predictions)):
    print(ids[i], predictions[i])
    file.write(str(ids[i]) + "," + str(predictions[i]) + "\n")

1461 129306.664
1462 168759.14
1463 195549.5
1464 186745.27
1465 185448.36
1466 173631.1
1467 187677.52
1468 165947.55
1469 168836.36
1470 118670.89
1471 203917.14
1472 97717.79
1473 102557.04
1474 154544.78
1475 121550.45
1476 386925.7
1477 256213.19
1478 303350.8
1479 253493.4
1480 500189.62
1481 281329.56
1482 202227.9
1483 159712.16
1484 165079.7
1485 166258.25
1486 202967.22
1487 328860.62
1488 237498.39
1489 219977.89
1490 223419.94
1491 177980.97
1492 89340.32
1493 212899.4
1494 303360.78
1495 283707.0
1496 213628.66
1497 171630.17
1498 156024.05
1499 147954.3
1500 145891.52
1501 179129.55
1502 135392.25
1503 328130.6
1504 230735.94
1505 227977.89
1506 171921.12
1507 255545.73
1508 185466.48
1509 161467.56
1510 153930.94
1511 145532.16
1512 163870.95
1513 145079.5
1514 202897.02
1515 207062.11
1516 148960.73
1517 178438.12
1518 112127.45
1519 240292.2
1520 147477.31
1521 144113.67
1522 166664.86
1523 107224.7
1524 108825.58
1525 125570.91
1526 135858.53
1527 91380.84
1528 139849